
<a id='whatsnew-0190'></a>

# v0.19.0 (October 2, 2016)

{{ header }}

This is a major release from 0.18.1 and includes number of API changes, several new features,
enhancements, and performance improvements along with a large number of bug fixes. We recommend that all
users upgrade to this version.

Highlights include:

- `merge_asof()` for asof-style time-series joining, see [here](#whatsnew-0190-enhancements-asof-merge)  
- `.rolling()` is now time-series aware, see [here](#whatsnew-0190-enhancements-rolling-ts)  
- `read_csv()` now supports parsing `Categorical` data, see [here](#whatsnew-0190-enhancements-read-csv-categorical)  
- A function `union_categorical()` has been added for combining categoricals, see [here](#whatsnew-0190-enhancements-union-categoricals)  
- `PeriodIndex` now has its own `period` dtype, and changed to be more consistent with other `Index` classes. See [here](#whatsnew-0190-api-period)  
- Sparse data structures gained enhanced support of `int` and `bool` dtypes, see [here](#whatsnew-0190-sparse)  
- Comparison operations with `Series` no longer ignores the index, see [here](#whatsnew-0190-api-series-ops) for an overview of the API changes.  
- Introduction of a pandas development API for utility functions, see [here](#whatsnew-0190-dev-api).  
- Deprecation of `Panel4D` and `PanelND`. We recommend to represent these types of n-dimensional data with the [xarray package](http://xarray.pydata.org/en/stable/).  
- Removal of the previously deprecated modules `pandas.io.data`, `pandas.io.wb`, `pandas.tools.rplot`.  


pandas >= 0.19.0 will no longer silence numpy ufunc warnings upon import, see [here](#whatsnew-0190-errstate).

## What’s new in v0.19.0

- [New features](#New-features)  
  - [`merge_asof` for asof-style time-series joining](#`merge_asof`-for-asof-style-time-series-joining)  
  - [`.rolling()` is now time-series aware](#`.rolling()`-is-now-time-series-aware)  
  - [`read_csv` has improved support for duplicate column names](#`read_csv`-has-improved-support-for-duplicate-column-names)  
  - [`read_csv` supports parsing `Categorical` directly](#`read_csv`-supports-parsing-`Categorical`-directly)  
  - [Categorical concatenation](#Categorical-concatenation)  
  - [Semi-month offsets](#Semi-month-offsets)  
  - [New Index methods](#New-Index-methods)  
  - [Google BigQuery Enhancements](#Google-BigQuery-Enhancements)  
  - [Fine-grained numpy errstate](#Fine-grained-numpy-errstate)  
  - [`get_dummies` now returns integer dtypes](#`get_dummies`-now-returns-integer-dtypes)  
  - [Downcast values to smallest possible dtype in `to_numeric`](#Downcast-values-to-smallest-possible-dtype-in-`to_numeric`)  
  - [pandas development API](#pandas-development-API)  
  - [Other enhancements](#Other-enhancements)  
- [API changes](#API-changes)  
  - [`Series.tolist()` will now return Python types](#`Series.tolist()`-will-now-return-Python-types)  
  - [`Series` operators for different indexes](#`Series`-operators-for-different-indexes)  
    - [Arithmetic operators](#Arithmetic-operators)  
    - [Comparison operators](#Comparison-operators)  
    - [Logical operators](#Logical-operators)  
    - [Flexible comparison methods](#Flexible-comparison-methods)  
  - [`Series` type promotion on assignment](#`Series`-type-promotion-on-assignment)  
  - [`.to_datetime()` changes](#`.to_datetime()`-changes)  
  - [Merging changes](#Merging-changes)  
  - [`.describe()` changes](#`.describe()`-changes)  
  - [`Period` changes](#`Period`-changes)  
    - [`PeriodIndex` now has `period` dtype](#`PeriodIndex`-now-has-`period`-dtype)  
    - [`Period('NaT')` now returns `pd.NaT`](#`Period('NaT')`-now-returns-`pd.NaT`)  
    - [`PeriodIndex.values` now returns array of `Period` object](#`PeriodIndex.values`-now-returns-array-of-`Period`-object)  
  - [Index `+` / `-` no longer used for set operations](#Index-`+`-/-`-`-no-longer-used-for-set-operations)  
  - [`Index.difference` and `.symmetric_difference` changes](#`Index.difference`-and-`.symmetric_difference`-changes)  
  - [`Index.unique` consistently returns `Index`](#`Index.unique`-consistently-returns-`Index`)  
  - [`MultiIndex` constructors, `groupby` and `set_index` preserve categorical dtypes](#`MultiIndex`-constructors,-`groupby`-and-`set_index`-preserve-categorical-dtypes)  
  - [`read_csv` will progressively enumerate chunks](#`read_csv`-will-progressively-enumerate-chunks)  
  - [Sparse Changes](#Sparse-Changes)  
    - [`int64` and `bool` support enhancements](#`int64`-and-`bool`-support-enhancements)  
    - [Operators now preserve dtypes](#Operators-now-preserve-dtypes)  
    - [Other sparse fixes](#Other-sparse-fixes)  
  - [Indexer dtype changes](#Indexer-dtype-changes)  
  - [Other API changes](#Other-API-changes)  
- [Deprecations](#Deprecations)  
- [Removal of prior version deprecations/changes](#Removal-of-prior-version-deprecations/changes)  
- [Performance improvements](#Performance-improvements)  
- [Bug fixes](#Bug-fixes)  
- [Contributors](#Contributors)  


<a id='whatsnew-0190-new-features'></a>

## New features


<a id='whatsnew-0190-enhancements-asof-merge'></a>

### `merge_asof` for asof-style time-series joining

A long-time requested feature has been added through the `merge_asof()` function, to
support asof style joining of time-series ([GH1870](https://github.com/pandas-dev/pandas/issues/1870), [GH13695](https://github.com/pandas-dev/pandas/issues/13695), [GH13709](https://github.com/pandas-dev/pandas/issues/13709), [GH13902](https://github.com/pandas-dev/pandas/issues/13902)). Full documentation is
[here](user_guide/merging.ipynb#merging-merge-asof).

The `merge_asof()` performs an asof merge, which is similar to a left-join
except that we match on nearest key rather than equal keys.

We typically want to match exactly when possible, and use the most
recent value otherwise.

We can also match rows ONLY with prior data, and not an exact match.

In a typical time-series example, we have `trades` and `quotes` and we want to `asof-join` them.
This also illustrates using the `by` parameter to group data before merging.

An asof merge joins on the `on`, typically a datetimelike field, which is ordered, and
in this case we are using a grouper in the `by` field. This is like a left-outer join, except
that forward filling happens automatically taking the most recent non-NaN value.

This returns a merged DataFrame with the entries in the same order as the original left
passed DataFrame (`trades` in this case), with the fields of the `quotes` merged.


<a id='whatsnew-0190-enhancements-rolling-ts'></a>

### `.rolling()` is now time-series aware

`.rolling()` objects are now time-series aware and can accept a time-series offset (or convertible) for the `window` argument ([GH13327](https://github.com/pandas-dev/pandas/issues/13327), [GH12995](https://github.com/pandas-dev/pandas/issues/12995)).
See the full documentation [here](user_guide/computation.ipynb#stats-moments-ts).

This is a regular frequency index. Using an integer window parameter works to roll along the window frequency.

Specifying an offset allows a more intuitive specification of the rolling frequency.

Using a non-regular, but still monotonic index, rolling with an integer window does not impart any special calculation.

Using the time-specification generates variable windows for this sparse data.

Furthermore, we now allow an optional `on` parameter to specify a column (rather than the
default of the index) in a DataFrame.


<a id='whatsnew-0190-enhancements-read-csv-dupe-col-names-support'></a>

### `read_csv` has improved support for duplicate column names

[Duplicate column names](user_guide/io.ipynb#io-dupe-names) are now supported in `read_csv()` whether
they are in the file or passed in as the `names` parameter ([GH7160](https://github.com/pandas-dev/pandas/issues/7160), [GH9424](https://github.com/pandas-dev/pandas/issues/9424))

**Previous behavior**:

```ipython
In [2]: pd.read_csv(StringIO(data), names=names)
Out[2]:
   a  b  a
0  2  1  2
1  5  4  5
```


The first `a` column contained the same data as the second `a` column, when it should have
contained the values `[0, 3]`.

**New behavior**:


<a id='whatsnew-0190-enhancements-read-csv-categorical'></a>

### `read_csv` supports parsing `Categorical` directly

The `read_csv()` function now supports parsing a `Categorical` column when
specified as a dtype ([GH10153](https://github.com/pandas-dev/pandas/issues/10153)).  Depending on the structure of the data,
this can result in a faster parse time and lower memory usage compared to
converting to `Categorical` after parsing.  See the io [docs here](user_guide/io.ipynb#io-categorical).

Individual columns can be parsed as a `Categorical` using a dict specification

>**Note**
>
>The resulting categories will always be parsed as strings (object dtype).
If the categories are numeric they can be converted using the
`to_numeric()` function, or as appropriate, another converter
such as `to_datetime()`.


<a id='whatsnew-0190-enhancements-union-categoricals'></a>

### Categorical concatenation

- A function `union_categoricals()` has been added for combining categoricals, see [Unioning Categoricals](user_guide/categorical.ipynb#categorical-union) ([GH13361](https://github.com/pandas-dev/pandas/issues/13361), [GH13763](https://github.com/pandas-dev/pandas/issues/13763), [GH13846](https://github.com/pandas-dev/pandas/issues/13846), [GH14173](https://github.com/pandas-dev/pandas/issues/14173))  
- `concat` and `append` now can concat `category` dtypes with different `categories` as `object` dtype ([GH13524](https://github.com/pandas-dev/pandas/issues/13524))  


**Previous behavior**:

```ipython
In [1]: pd.concat([s1, s2])
ValueError: incompatible categories in categorical concat
```


**New behavior**:


<a id='whatsnew-0190-enhancements-semi-month-offsets'></a>

### Semi-month offsets

Pandas has gained new frequency offsets, `SemiMonthEnd` (‘SM’) and `SemiMonthBegin` (‘SMS’).
These provide date offsets anchored (by default) to the 15th and end of month, and 15th and 1st of month respectively.
([GH1543](https://github.com/pandas-dev/pandas/issues/1543))

**SemiMonthEnd**:

**SemiMonthBegin**:

Using the anchoring suffix, you can also specify the day of month to use instead of the 15th.


<a id='whatsnew-0190-enhancements-index'></a>

### New Index methods

The following methods and options are added to `Index`, to be more consistent with the `Series` and `DataFrame` API.

`Index` now supports the `.where()` function for same shape indexing ([GH13170](https://github.com/pandas-dev/pandas/issues/13170))

`Index` now supports `.dropna()` to exclude missing values ([GH6194](https://github.com/pandas-dev/pandas/issues/6194))

For `MultiIndex`, values are dropped if any level is missing by default. Specifying
`how='all'` only drops values where all levels are missing.

`Index` now supports `.str.extractall()` which returns a `DataFrame`, see the [docs here](user_guide/text.ipynb#text-extractall) ([GH10008](https://github.com/pandas-dev/pandas/issues/10008), [GH13156](https://github.com/pandas-dev/pandas/issues/13156))

`Index.astype()` now accepts an optional boolean argument `copy`, which allows optional copying if the requirements on dtype are satisfied ([GH13209](https://github.com/pandas-dev/pandas/issues/13209))


<a id='whatsnew-0190-gbq'></a>

### Google BigQuery Enhancements

- The `read_gbq()` method has gained the `dialect` argument to allow users to specify whether to use BigQuery’s legacy SQL or BigQuery’s standard SQL. See the [docs](https://pandas-gbq.readthedocs.io/en/latest/reading.html) for more details ([GH13615](https://github.com/pandas-dev/pandas/issues/13615)).  
- The `to_gbq()` method now allows the DataFrame column order to differ from the destination table schema ([GH11359](https://github.com/pandas-dev/pandas/issues/11359)).  



<a id='whatsnew-0190-errstate'></a>

### Fine-grained numpy errstate

Previous versions of pandas would permanently silence numpy’s ufunc error handling when `pandas` was imported. Pandas did this in order to silence the warnings that would arise from using numpy ufuncs on missing data, which are usually represented as `NaN` s. Unfortunately, this silenced legitimate warnings arising in non-pandas code in the application. Starting with 0.19.0, pandas will use the `numpy.errstate` context manager to silence these warnings in a more fine-grained manner, only around where these operations are actually used in the pandas code base. ([GH13109](https://github.com/pandas-dev/pandas/issues/13109), [GH13145](https://github.com/pandas-dev/pandas/issues/13145))

After upgrading pandas, you may see *new* `RuntimeWarnings` being issued from your code. These are likely legitimate, and the underlying cause likely existed in the code when using previous versions of pandas that simply silenced the warning. Use [numpy.errstate](http://docs.scipy.org/doc/numpy/reference/generated/numpy.errstate.html) around the source of the `RuntimeWarning` to control how these conditions are handled.


<a id='whatsnew-0190-get-dummies-dtypes'></a>

### `get_dummies` now returns integer dtypes

The `pd.get_dummies` function now returns dummy-encoded columns as small integers, rather than floats ([GH8725](https://github.com/pandas-dev/pandas/issues/8725)). This should provide an improved memory footprint.

**Previous behavior**:

```ipython
In [1]: pd.get_dummies(['a', 'b', 'a', 'c']).dtypes

Out[1]:
a    float64
b    float64
c    float64
dtype: object
```


**New behavior**:


<a id='whatsnew-0190-enhancements-to-numeric-downcast'></a>

### Downcast values to smallest possible dtype in `to_numeric`

`pd.to_numeric()` now accepts a `downcast` parameter, which will downcast the data if possible to smallest specified numerical dtype ([GH13352](https://github.com/pandas-dev/pandas/issues/13352))


<a id='whatsnew-0190-dev-api'></a>

### pandas development API

As part of making pandas API more uniform and accessible in the future, we have created a standard
sub-package of pandas, `pandas.api` to hold public API’s. We are starting by exposing type
introspection functions in `pandas.api.types`. More sub-packages and officially sanctioned API’s
will be published in future versions of pandas ([GH13147](https://github.com/pandas-dev/pandas/issues/13147), [GH13634](https://github.com/pandas-dev/pandas/issues/13634))

The following are now part of this API:

>**Note**
>
>Calling these functions from the internal module `pandas.core.common` will now show a `DeprecationWarning` ([GH13990](https://github.com/pandas-dev/pandas/issues/13990))


<a id='whatsnew-0190-enhancements-other'></a>

### Other enhancements

- `Timestamp` can now accept positional and keyword parameters similar to `datetime.datetime()` ([GH10758](https://github.com/pandas-dev/pandas/issues/10758), [GH11630](https://github.com/pandas-dev/pandas/issues/11630))  
- The `.resample()` function now accepts a `on=` or `level=` parameter for resampling on a datetimelike column or `MultiIndex` level ([GH13500](https://github.com/pandas-dev/pandas/issues/13500))  
- The `.get_credentials()` method of `GbqConnector` can now first try to fetch [the application default credentials](https://developers.google.com/identity/protocols/application-default-credentials). See the docs for more details ([GH13577](https://github.com/pandas-dev/pandas/issues/13577)).  
- The `.tz_localize()` method of `DatetimeIndex` and `Timestamp` has gained the `errors` keyword, so you can potentially coerce nonexistent timestamps to `NaT`. The default behavior remains to raising a `NonExistentTimeError` ([GH13057](https://github.com/pandas-dev/pandas/issues/13057))  
- `.to_hdf/read_hdf()` now accept path objects (e.g. `pathlib.Path`, `py.path.local`) for the file path ([GH11773](https://github.com/pandas-dev/pandas/issues/11773))  
- The `pd.read_csv()` with `engine='python'` has gained support for the
  `decimal` ([GH12933](https://github.com/pandas-dev/pandas/issues/12933)), `na_filter` ([GH13321](https://github.com/pandas-dev/pandas/issues/13321)) and the `memory_map` option ([GH13381](https://github.com/pandas-dev/pandas/issues/13381)).  
- Consistent with the Python API, `pd.read_csv()` will now interpret `+inf` as positive infinity ([GH13274](https://github.com/pandas-dev/pandas/issues/13274))  
- The `pd.read_html()` has gained support for the `na_values`, `converters`, `keep_default_na`  options ([GH13461](https://github.com/pandas-dev/pandas/issues/13461))  
- `Categorical.astype()` now accepts an optional boolean argument `copy`, effective when dtype is categorical ([GH13209](https://github.com/pandas-dev/pandas/issues/13209))  
- `DataFrame` has gained the `.asof()` method to return the last non-NaN values according to the selected subset ([GH13358](https://github.com/pandas-dev/pandas/issues/13358))  
- The `DataFrame` constructor will now respect key ordering if a list of `OrderedDict` objects are passed in ([GH13304](https://github.com/pandas-dev/pandas/issues/13304))  
- `pd.read_html()` has gained support for the `decimal` option ([GH12907](https://github.com/pandas-dev/pandas/issues/12907))  
- `Series` has gained the properties `.is_monotonic`, `.is_monotonic_increasing`, `.is_monotonic_decreasing`, similar to `Index` ([GH13336](https://github.com/pandas-dev/pandas/issues/13336))  
- `DataFrame.to_sql()` now allows a single value as the SQL type for all columns ([GH11886](https://github.com/pandas-dev/pandas/issues/11886)).  
- `Series.append` now supports the `ignore_index` option ([GH13677](https://github.com/pandas-dev/pandas/issues/13677))  
- `.to_stata()` and `StataWriter` can now write variable labels to Stata dta files using a dictionary to make column names to labels ([GH13535](https://github.com/pandas-dev/pandas/issues/13535), [GH13536](https://github.com/pandas-dev/pandas/issues/13536))  
- `.to_stata()` and `StataWriter` will automatically convert `datetime64[ns]` columns to Stata format `%tc`, rather than raising a `ValueError` ([GH12259](https://github.com/pandas-dev/pandas/issues/12259))  
- `read_stata()` and `StataReader` raise with a more explicit error message when reading Stata files with repeated value labels when `convert_categoricals=True` ([GH13923](https://github.com/pandas-dev/pandas/issues/13923))  
- `DataFrame.style` will now render sparsified MultiIndexes ([GH11655](https://github.com/pandas-dev/pandas/issues/11655))  
- `DataFrame.style` will now show column level names (e.g. `DataFrame.columns.names`) ([GH13775](https://github.com/pandas-dev/pandas/issues/13775))  
- `DataFrame` has gained support to re-order the columns based on the values
  in a row using `df.sort_values(by='...', axis=1)` ([GH10806](https://github.com/pandas-dev/pandas/issues/10806))  
- Added documentation to [I/O](user_guide/io.ipynb#io-dtypes) regarding the perils of reading in columns with mixed dtypes and how to handle it ([GH13746](https://github.com/pandas-dev/pandas/issues/13746))  
- `to_html()` now has a `border` argument to control the value in the opening `<table>` tag. The default is the value of the `html.border` option, which defaults to 1. This also affects the notebook HTML repr, but since Jupyter’s CSS includes a border-width attribute, the visual effect is the same. ([GH11563](https://github.com/pandas-dev/pandas/issues/11563)).  
- Raise `ImportError` in the sql functions when `sqlalchemy` is not installed and a connection string is used ([GH11920](https://github.com/pandas-dev/pandas/issues/11920)).  
- Compatibility with matplotlib 2.0. Older versions of pandas should also work with matplotlib 2.0 ([GH13333](https://github.com/pandas-dev/pandas/issues/13333))  
- `Timestamp`, `Period`, `DatetimeIndex`, `PeriodIndex` and `.dt` accessor have gained a `.is_leap_year` property to check whether the date belongs to a leap year. ([GH13727](https://github.com/pandas-dev/pandas/issues/13727))  
- `astype()` will now accept a dict of column name to data types mapping as the `dtype` argument. ([GH12086](https://github.com/pandas-dev/pandas/issues/12086))  
- The `pd.read_json` and `DataFrame.to_json` has gained support for reading and writing json lines with `lines` option see [Line delimited json](user_guide/io.ipynb#io-jsonl) ([GH9180](https://github.com/pandas-dev/pandas/issues/9180))  
- `read_excel()` now supports the true_values and false_values keyword arguments ([GH13347](https://github.com/pandas-dev/pandas/issues/13347))  
- `groupby()` will now accept a scalar and a single-element list for specifying `level` on a non-`MultiIndex` grouper. ([GH13907](https://github.com/pandas-dev/pandas/issues/13907))  
- Non-convertible dates in an excel date column will be returned without conversion and the column will be `object` dtype, rather than raising an exception ([GH10001](https://github.com/pandas-dev/pandas/issues/10001)).  
- `pd.Timedelta(None)` is now accepted and will return `NaT`, mirroring `pd.Timestamp` ([GH13687](https://github.com/pandas-dev/pandas/issues/13687))  
- `pd.read_stata()` can now handle some format 111 files, which are produced by SAS when generating Stata dta files ([GH11526](https://github.com/pandas-dev/pandas/issues/11526))  
- `Series` and `Index` now support `divmod` which will return a tuple of
  series or indices. This behaves like a standard binary operator with regards
  to broadcasting rules ([GH14208](https://github.com/pandas-dev/pandas/issues/14208)).  



<a id='whatsnew-0190-api'></a>

## API changes

### `Series.tolist()` will now return Python types

`Series.tolist()` will now return Python types in the output, mimicking NumPy `.tolist()` behavior ([GH10904](https://github.com/pandas-dev/pandas/issues/10904))

**Previous behavior**:

```ipython
In [7]: type(s.tolist()[0])
Out[7]:
 <class 'numpy.int64'>
```


**New behavior**:


<a id='whatsnew-0190-api-series-ops'></a>

### `Series` operators for different indexes

Following `Series` operators have been changed to make all operators consistent,
including `DataFrame` ([GH1134](https://github.com/pandas-dev/pandas/issues/1134), [GH4581](https://github.com/pandas-dev/pandas/issues/4581), [GH13538](https://github.com/pandas-dev/pandas/issues/13538))

- `Series` comparison operators now raise `ValueError` when `index` are different.  
- `Series` logical operators align both `index` of left and right hand side.  


Until 0.18.1, comparing `Series` with the same length, would succeed even if
the `.index` are different (the result ignores `.index`). As of 0.19.0, this will raises `ValueError` to be more strict. This section also describes how to keep previous behavior or align different indexes, using the flexible comparison methods like `.eq`.

As a result, `Series` and `DataFrame` operators behave as below:

#### Arithmetic operators

Arithmetic operators align both `index` (no changes).

#### Comparison operators

Comparison operators raise `ValueError` when `.index` are different.

**Previous behavior** (`Series`):

`Series` compared values ignoring the `.index` as long as both had the same length:

```ipython
In [1]: s1 == s2
Out[1]:
A    False
B     True
C    False
dtype: bool
```


**New behavior** (`Series`):

```ipython
In [2]: s1 == s2
Out[2]:
ValueError: Can only compare identically-labeled Series objects
```


>**Note**
>
>To achieve the same result as previous versions (compare values based on locations ignoring `.index`), compare both `.values`.

If you want to compare `Series` aligning its `.index`, see flexible comparison methods section below:

**Current behavior** (`DataFrame`, no change):

```ipython
In [3]: df1 == df2
Out[3]:
ValueError: Can only compare identically-labeled DataFrame objects
```


#### Logical operators

Logical operators align both `.index` of left and right hand side.

**Previous behavior** (`Series`), only left hand side `index` was kept:

```ipython
In [4]: s1 = pd.Series([True, False, True], index=list('ABC'))
In [5]: s2 = pd.Series([True, True, True], index=list('ABD'))
In [6]: s1 & s2
Out[6]:
A     True
B    False
C    False
dtype: bool
```


**New behavior** (`Series`):

>**Note**
>
>`Series` logical operators fill a `NaN` result with `False`.

>**Note**
>
>To achieve the same result as previous versions (compare values based on only left hand side index), you can use `reindex_like`:

**Current behavior** (`DataFrame`, no change):

#### Flexible comparison methods

`Series` flexible comparison methods like `eq`, `ne`, `le`, `lt`, `ge` and `gt` now align both `index`. Use these operators if you want to compare two `Series`
which has the different `index`.

Previously, this worked the same as comparison operators (see above).


<a id='whatsnew-0190-api-promote'></a>

### `Series` type promotion on assignment

A `Series` will now correctly promote its dtype for assignment with incompat values to the current dtype ([GH13234](https://github.com/pandas-dev/pandas/issues/13234))

**Previous behavior**:

```ipython
In [2]: s["a"] = pd.Timestamp("2016-01-01")

In [3]: s["b"] = 3.0
TypeError: invalid type promotion
```


**New behavior**:


<a id='whatsnew-0190-api-to-datetime-coerce'></a>

### `.to_datetime()` changes

Previously if `.to_datetime()` encountered mixed integers/floats and strings, but no datetimes with `errors='coerce'` it would convert all to `NaT`.

**Previous behavior**:

```ipython
In [2]: pd.to_datetime([1, 'foo'], errors='coerce')
Out[2]: DatetimeIndex(['NaT', 'NaT'], dtype='datetime64[ns]', freq=None)
```


**Current behavior**:

This will now convert integers/floats with the default unit of `ns`.

Bug fixes related to `.to_datetime()`:

- Bug in `pd.to_datetime()` when passing integers or floats, and no `unit` and `errors='coerce'` ([GH13180](https://github.com/pandas-dev/pandas/issues/13180)).  
- Bug in `pd.to_datetime()` when passing invalid data types (e.g. bool); will now respect the `errors` keyword ([GH13176](https://github.com/pandas-dev/pandas/issues/13176))  
- Bug in `pd.to_datetime()` which overflowed on `int8`, and `int16` dtypes ([GH13451](https://github.com/pandas-dev/pandas/issues/13451))  
- Bug in `pd.to_datetime()` raise `AttributeError` with `NaN` and the other string is not valid when `errors='ignore'` ([GH12424](https://github.com/pandas-dev/pandas/issues/12424))  
- Bug in `pd.to_datetime()` did not cast floats correctly when `unit` was specified, resulting in truncated datetime ([GH13834](https://github.com/pandas-dev/pandas/issues/13834))  



<a id='whatsnew-0190-api-merging'></a>

### Merging changes

Merging will now preserve the dtype of the join keys ([GH8596](https://github.com/pandas-dev/pandas/issues/8596))

**Previous behavior**:

```ipython
In [5]: pd.merge(df1, df2, how='outer')
Out[5]:
   key    v1
0  1.0  10.0
1  1.0  20.0
2  2.0  30.0

In [6]: pd.merge(df1, df2, how='outer').dtypes
Out[6]:
key    float64
v1     float64
dtype: object
```


**New behavior**:

We are able to preserve the join keys

Of course if you have missing values that are introduced, then the
resulting dtype will be upcast, which is unchanged from previous.


<a id='whatsnew-0190-api-describe'></a>

### `.describe()` changes

Percentile identifiers in the index of a `.describe()` output will now be rounded to the least precision that keeps them distinct ([GH13104](https://github.com/pandas-dev/pandas/issues/13104))

**Previous behavior**:

The percentiles were rounded to at most one decimal place, which could raise `ValueError` for a data frame if the percentiles were duplicated.

```ipython
In [3]: s.describe(percentiles=[0.0001, 0.0005, 0.001, 0.999, 0.9995, 0.9999])
Out[3]:
count     5.000000
mean      2.000000
std       1.581139
min       0.000000
0.0%      0.000400
0.1%      0.002000
0.1%      0.004000
50%       2.000000
99.9%     3.996000
100.0%    3.998000
100.0%    3.999600
max       4.000000
dtype: float64

In [4]: df.describe(percentiles=[0.0001, 0.0005, 0.001, 0.999, 0.9995, 0.9999])
Out[4]:
...
ValueError: cannot reindex from a duplicate axis
```


**New behavior**:

Furthermore:

- Passing duplicated `percentiles` will now raise a `ValueError`.  
- Bug in `.describe()` on a DataFrame with a mixed-dtype column index, which would previously raise a `TypeError` ([GH13288](https://github.com/pandas-dev/pandas/issues/13288))  



<a id='whatsnew-0190-api-period'></a>

### `Period` changes

#### `PeriodIndex` now has `period` dtype

`PeriodIndex` now has its own `period` dtype. The `period` dtype is a
pandas extension dtype like `category` or the [timezone aware dtype](user_guide/timeseries.ipynb#timeseries-timezone-series) (`datetime64[ns, tz]`) ([GH13941](https://github.com/pandas-dev/pandas/issues/13941)).
As a consequence of this change, `PeriodIndex` no longer has an integer dtype:

**Previous behavior**:

```ipython
In [1]: pi = pd.PeriodIndex(['2016-08-01'], freq='D')

In [2]: pi
Out[2]: PeriodIndex(['2016-08-01'], dtype='int64', freq='D')

In [3]: pd.api.types.is_integer_dtype(pi)
Out[3]: True

In [4]: pi.dtype
Out[4]: dtype('int64')
```


**New behavior**:


<a id='whatsnew-0190-api-periodnat'></a>

#### `Period('NaT')` now returns `pd.NaT`

Previously, `Period` has its own `Period('NaT')` representation different from `pd.NaT`. Now `Period('NaT')` has been changed to return `pd.NaT`. ([GH12759](https://github.com/pandas-dev/pandas/issues/12759), [GH13582](https://github.com/pandas-dev/pandas/issues/13582))

**Previous behavior**:

```ipython
In [5]: pd.Period('NaT', freq='D')
Out[5]: Period('NaT', 'D')
```


**New behavior**:

These result in `pd.NaT` without providing `freq` option.

To be compatible with `Period` addition and subtraction, `pd.NaT` now supports addition and subtraction with `int`. Previously it raised `ValueError`.

**Previous behavior**:

```ipython
In [5]: pd.NaT + 1
...
ValueError: Cannot add integral value to Timestamp without freq.
```


**New behavior**:

#### `PeriodIndex.values` now returns array of `Period` object

`.values` is changed to return an array of `Period` objects, rather than an array
of integers ([GH13988](https://github.com/pandas-dev/pandas/issues/13988)).

**Previous behavior**:

```ipython
In [6]: pi = pd.PeriodIndex(['2011-01', '2011-02'], freq='M')
In [7]: pi.values
Out[7]: array([492, 493])
```


**New behavior**:


<a id='whatsnew-0190-api-setops'></a>

### Index `+` / `-` no longer used for set operations

Addition and subtraction of the base Index type and of DatetimeIndex
(not the numeric index types)
previously performed set operations (set union and difference). This
behavior was already deprecated since 0.15.0 (in favor using the specific
`.union()` and `.difference()` methods), and is now disabled. When
possible, `+` and `-` are now used for element-wise operations, for
example for concatenating strings or subtracting datetimes
([GH8227](https://github.com/pandas-dev/pandas/issues/8227), [GH14127](https://github.com/pandas-dev/pandas/issues/14127)).

Previous behavior:

```ipython
In [1]: pd.Index(['a', 'b']) + pd.Index(['a', 'c'])
FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()
Out[1]: Index(['a', 'b', 'c'], dtype='object')
```


**New behavior**: the same operation will now perform element-wise addition:

Note that numeric Index objects already performed element-wise operations.
For example, the behavior of adding two integer Indexes is unchanged.
The base `Index` is now made consistent with this behavior.

Further, because of this change, it is now possible to subtract two
DatetimeIndex objects resulting in a TimedeltaIndex:

**Previous behavior**:

```ipython
In [1]: (pd.DatetimeIndex(['2016-01-01', '2016-01-02'])
   ...:  - pd.DatetimeIndex(['2016-01-02', '2016-01-03']))
FutureWarning: using '-' to provide set differences with datetimelike Indexes is deprecated, use .difference()
Out[1]: DatetimeIndex(['2016-01-01'], dtype='datetime64[ns]', freq=None)
```


**New behavior**:


<a id='whatsnew-0190-api-difference'></a>

### `Index.difference` and `.symmetric_difference` changes

`Index.difference` and `Index.symmetric_difference` will now, more consistently, treat `NaN` values as any other values. ([GH13514](https://github.com/pandas-dev/pandas/issues/13514))

**Previous behavior**:

```ipython
In [3]: idx1.difference(idx2)
Out[3]: Float64Index([nan, 2.0, 3.0], dtype='float64')

In [4]: idx1.symmetric_difference(idx2)
Out[4]: Float64Index([0.0, nan, 2.0, 3.0], dtype='float64')
```


**New behavior**:


<a id='whatsnew-0190-api-unique-index'></a>

### `Index.unique` consistently returns `Index`

`Index.unique()` now returns unique values as an
`Index` of the appropriate `dtype`. ([GH13395](https://github.com/pandas-dev/pandas/issues/13395)).
Previously, most `Index` classes returned `np.ndarray`, and `DatetimeIndex`,
`TimedeltaIndex` and `PeriodIndex` returned `Index` to keep metadata like timezone.

**Previous behavior**:

```ipython
In [1]: pd.Index([1, 2, 3]).unique()
Out[1]: array([1, 2, 3])

In [2]: pd.DatetimeIndex(['2011-01-01', '2011-01-02',
   ...:                   '2011-01-03'], tz='Asia/Tokyo').unique()
Out[2]:
DatetimeIndex(['2011-01-01 00:00:00+09:00', '2011-01-02 00:00:00+09:00',
               '2011-01-03 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Tokyo]', freq=None)
```


**New behavior**:


<a id='whatsnew-0190-api-multiindex'></a>

### `MultiIndex` constructors, `groupby` and `set_index` preserve categorical dtypes

`MultiIndex.from_arrays` and `MultiIndex.from_product` will now preserve categorical dtype
in `MultiIndex` levels ([GH13743](https://github.com/pandas-dev/pandas/issues/13743), [GH13854](https://github.com/pandas-dev/pandas/issues/13854)).

**Previous behavior**:

```ipython
In [4]: midx.levels[0]
Out[4]: Index(['b', 'a', 'c'], dtype='object')

In [5]: midx.get_level_values[0]
Out[5]: Index(['a', 'b'], dtype='object')
```


**New behavior**: the single level is now a `CategoricalIndex`:

An analogous change has been made to `MultiIndex.from_product`.
As a consequence, `groupby` and `set_index` also preserve categorical dtypes in indexes

**Previous behavior**:

```ipython
In [11]: df_grouped.index.levels[1]
Out[11]: Index(['b', 'a', 'c'], dtype='object', name='C')
In [12]: df_grouped.reset_index().dtypes
Out[12]:
A      int64
C     object
B    float64
dtype: object

In [13]: df_set_idx.index.levels[1]
Out[13]: Index(['b', 'a', 'c'], dtype='object', name='C')
In [14]: df_set_idx.reset_index().dtypes
Out[14]:
A      int64
C     object
B      int64
dtype: object
```


**New behavior**:


<a id='whatsnew-0190-api-autogenerated-chunksize-index'></a>

### `read_csv` will progressively enumerate chunks

When `read_csv()` is called with `chunksize=n` and without specifying an index,
each chunk used to have an independently generated index from `0` to `n-1`.
They are now given instead a progressive index, starting from `0` for the first chunk,
from `n` for the second, and so on, so that, when concatenated, they are identical to
the result of calling `read_csv()` without the `chunksize=` argument
([GH12185](https://github.com/pandas-dev/pandas/issues/12185)).

**Previous behavior**:

```ipython
In [2]: pd.concat(pd.read_csv(StringIO(data), chunksize=2))
Out[2]:
   A  B
0  0  1
1  2  3
0  4  5
1  6  7
```


**New behavior**:


<a id='whatsnew-0190-sparse'></a>

### Sparse Changes

These changes allow pandas to handle sparse data with more dtypes, and for work to make a smoother experience with data handling.

#### `int64` and `bool` support enhancements

Sparse data structures now gained enhanced support of `int64` and `bool` `dtype` ([GH667](https://github.com/pandas-dev/pandas/issues/667), [GH13849](https://github.com/pandas-dev/pandas/issues/13849)).

Previously, sparse data were `float64` dtype by default, even if all inputs were of `int` or `bool` dtype. You had to specify `dtype` explicitly to create sparse data with `int64` dtype. Also, `fill_value` had to be specified explicitly because the default was `np.nan` which doesn’t appear in `int64` or `bool` data.

```ipython
In [1]: pd.SparseArray([1, 2, 0, 0])
Out[1]:
[1.0, 2.0, 0.0, 0.0]
Fill: nan
IntIndex
Indices: array([0, 1, 2, 3], dtype=int32)

# specifying int64 dtype, but all values are stored in sp_values because
# fill_value default is np.nan
In [2]: pd.SparseArray([1, 2, 0, 0], dtype=np.int64)
Out[2]:
[1, 2, 0, 0]
Fill: nan
IntIndex
Indices: array([0, 1, 2, 3], dtype=int32)

In [3]: pd.SparseArray([1, 2, 0, 0], dtype=np.int64, fill_value=0)
Out[3]:
[1, 2, 0, 0]
Fill: 0
IntIndex
Indices: array([0, 1], dtype=int32)
```


As of v0.19.0, sparse data keeps the input dtype, and uses more appropriate `fill_value` defaults (`0` for `int64` dtype, `False` for `bool` dtype).

See the [docs](user_guide/sparse.ipynb#sparse-dtype) for more details.

#### Operators now preserve dtypes

- Sparse data structure now can preserve `dtype` after arithmetic ops ([GH13848](https://github.com/pandas-dev/pandas/issues/13848))  


- Sparse data structure now support `astype` to convert internal `dtype` ([GH13900](https://github.com/pandas-dev/pandas/issues/13900))  


`astype` fails if data contains values which cannot be converted to specified `dtype`.
Note that the limitation is applied to `fill_value` which default is `np.nan`.

```ipython
In [7]: pd.SparseSeries([1., np.nan, 2., np.nan], fill_value=np.nan).astype(np.int64)
Out[7]:
ValueError: unable to coerce current fill_value nan to int64 dtype
```


#### Other sparse fixes

- Subclassed `SparseDataFrame` and `SparseSeries` now preserve class types when slicing or transposing. ([GH13787](https://github.com/pandas-dev/pandas/issues/13787))  
- `SparseArray` with `bool` dtype now supports logical (bool) operators ([GH14000](https://github.com/pandas-dev/pandas/issues/14000))  
- Bug in `SparseSeries` with `MultiIndex` `[]` indexing may raise `IndexError` ([GH13144](https://github.com/pandas-dev/pandas/issues/13144))  
- Bug in `SparseSeries` with `MultiIndex` `[]` indexing result may have normal `Index` ([GH13144](https://github.com/pandas-dev/pandas/issues/13144))  
- Bug in `SparseDataFrame` in which `axis=None` did not default to `axis=0` ([GH13048](https://github.com/pandas-dev/pandas/issues/13048))  
- Bug in `SparseSeries` and `SparseDataFrame` creation with `object` dtype may raise `TypeError` ([GH11633](https://github.com/pandas-dev/pandas/issues/11633))  
- Bug in `SparseDataFrame` doesn’t respect passed `SparseArray` or `SparseSeries` ‘s dtype and `fill_value`  ([GH13866](https://github.com/pandas-dev/pandas/issues/13866))  
- Bug in `SparseArray` and `SparseSeries` don’t apply ufunc to `fill_value` ([GH13853](https://github.com/pandas-dev/pandas/issues/13853))  
- Bug in `SparseSeries.abs` incorrectly keeps negative `fill_value` ([GH13853](https://github.com/pandas-dev/pandas/issues/13853))  
- Bug in single row slicing on multi-type `SparseDataFrame` s, types were previously forced to float ([GH13917](https://github.com/pandas-dev/pandas/issues/13917))  
- Bug in `SparseSeries` slicing changes integer dtype to float ([GH8292](https://github.com/pandas-dev/pandas/issues/8292))  
- Bug in `SparseDataFarme` comparison ops may raise `TypeError` ([GH13001](https://github.com/pandas-dev/pandas/issues/13001))  
- Bug in `SparseDataFarme.isnull` raises `ValueError` ([GH8276](https://github.com/pandas-dev/pandas/issues/8276))  
- Bug in `SparseSeries` representation with `bool` dtype may raise `IndexError` ([GH13110](https://github.com/pandas-dev/pandas/issues/13110))  
- Bug in `SparseSeries` and `SparseDataFrame` of `bool` or `int64` dtype may display its values like `float64` dtype ([GH13110](https://github.com/pandas-dev/pandas/issues/13110))  
- Bug in sparse indexing using `SparseArray` with `bool` dtype may return incorrect result  ([GH13985](https://github.com/pandas-dev/pandas/issues/13985))  
- Bug in `SparseArray` created from `SparseSeries` may lose `dtype` ([GH13999](https://github.com/pandas-dev/pandas/issues/13999))  
- Bug in `SparseSeries` comparison with dense returns normal `Series` rather than `SparseSeries` ([GH13999](https://github.com/pandas-dev/pandas/issues/13999))  



<a id='whatsnew-0190-indexer-dtype'></a>

### Indexer dtype changes

>**Note**
>
>This change only affects 64 bit python running on Windows, and only affects relatively advanced
indexing operations

Methods such as `Index.get_indexer` that return an indexer array, coerce that array to a “platform int”, so that it can be
directly used in 3rd party library operations like `numpy.take`.  Previously, a platform int was defined as `np.int_`
which corresponds to a C integer, but the correct type, and what is being used now, is `np.intp`, which corresponds
to the C integer size that can hold a pointer ([GH3033](https://github.com/pandas-dev/pandas/issues/3033), [GH13972](https://github.com/pandas-dev/pandas/issues/13972)).

These types are the same on many platform, but for 64 bit python on Windows,
`np.int_` is 32 bits, and `np.intp` is 64 bits.  Changing this behavior improves performance for many
operations on that platform.

**Previous behavior**:

```ipython
In [1]: i = pd.Index(['a', 'b', 'c'])

In [2]: i.get_indexer(['b', 'b', 'c']).dtype
Out[2]: dtype('int32')
```


**New behavior**:

```ipython
In [1]: i = pd.Index(['a', 'b', 'c'])

In [2]: i.get_indexer(['b', 'b', 'c']).dtype
Out[2]: dtype('int64')
```



<a id='whatsnew-0190-api-other'></a>

### Other API changes

- `Timestamp.to_pydatetime` will issue a `UserWarning` when `warn=True`, and the instance has a non-zero number of nanoseconds, previously this would print a message to stdout ([GH14101](https://github.com/pandas-dev/pandas/issues/14101)).  
- `Series.unique()` with datetime and timezone now returns return array of `Timestamp` with timezone ([GH13565](https://github.com/pandas-dev/pandas/issues/13565)).  
- `Panel.to_sparse()` will raise a `NotImplementedError` exception when called ([GH13778](https://github.com/pandas-dev/pandas/issues/13778)).  
- `Index.reshape()` will raise a `NotImplementedError` exception when called ([GH12882](https://github.com/pandas-dev/pandas/issues/12882)).  
- `.filter()` enforces mutual exclusion of the keyword arguments ([GH12399](https://github.com/pandas-dev/pandas/issues/12399)).  
- `eval`’s upcasting rules for `float32` types have been updated to be more consistent with NumPy’s rules.  New behavior will not upcast to `float64` if you multiply a pandas `float32` object by a scalar float64 ([GH12388](https://github.com/pandas-dev/pandas/issues/12388)).  
- An `UnsupportedFunctionCall` error is now raised if NumPy ufuncs like `np.mean` are called on groupby or resample objects ([GH12811](https://github.com/pandas-dev/pandas/issues/12811)).  
- `__setitem__` will no longer apply a callable rhs as a function instead of storing it. Call `where` directly to get the previous behavior ([GH13299](https://github.com/pandas-dev/pandas/issues/13299)).  
- Calls to `.sample()` will respect the random seed set via `numpy.random.seed(n)` ([GH13161](https://github.com/pandas-dev/pandas/issues/13161))  
- `Styler.apply` is now more strict about the outputs your function must return. For `axis=0` or `axis=1`, the output shape must be identical. For `axis=None`, the output must be a DataFrame with identical columns and index labels ([GH13222](https://github.com/pandas-dev/pandas/issues/13222)).  
- `Float64Index.astype(int)` will now raise `ValueError` if `Float64Index` contains `NaN` values ([GH13149](https://github.com/pandas-dev/pandas/issues/13149))  
- `TimedeltaIndex.astype(int)` and `DatetimeIndex.astype(int)` will now return `Int64Index` instead of `np.array` ([GH13209](https://github.com/pandas-dev/pandas/issues/13209))  
- Passing `Period` with multiple frequencies to normal `Index` now returns `Index` with `object` dtype ([GH13664](https://github.com/pandas-dev/pandas/issues/13664))  
- `PeriodIndex.fillna` with `Period` has different freq now coerces to `object` dtype ([GH13664](https://github.com/pandas-dev/pandas/issues/13664))  
- Faceted boxplots from `DataFrame.boxplot(by=col)` now return a `Series` when `return_type` is not None. Previously these returned an `OrderedDict`. Note that when `return_type=None`, the default, these still return a 2-D NumPy array ([GH12216](https://github.com/pandas-dev/pandas/issues/12216), [GH7096](https://github.com/pandas-dev/pandas/issues/7096)).  
- `pd.read_hdf` will now raise a `ValueError` instead of `KeyError`, if a mode other than `r`, `r+` and `a` is supplied. ([GH13623](https://github.com/pandas-dev/pandas/issues/13623))  
- `pd.read_csv()`, `pd.read_table()`, and `pd.read_hdf()` raise the builtin `FileNotFoundError` exception for Python 3.x when called on a nonexistent file; this is back-ported as `IOError` in Python 2.x ([GH14086](https://github.com/pandas-dev/pandas/issues/14086))  
- More informative exceptions are passed through the csv parser. The exception type would now be the original exception type instead of `CParserError` ([GH13652](https://github.com/pandas-dev/pandas/issues/13652)).  
- `pd.read_csv()` in the C engine will now issue a `ParserWarning` or raise a `ValueError` when `sep` encoded is more than one character long ([GH14065](https://github.com/pandas-dev/pandas/issues/14065))  
- `DataFrame.values` will now return `float64` with a `DataFrame` of mixed `int64` and `uint64` dtypes, conforming to `np.find_common_type` ([GH10364](https://github.com/pandas-dev/pandas/issues/10364), [GH13917](https://github.com/pandas-dev/pandas/issues/13917))  
- `.groupby.groups` will now return a dictionary of `Index` objects, rather than a dictionary of `np.ndarray` or `lists` ([GH14293](https://github.com/pandas-dev/pandas/issues/14293))  



<a id='whatsnew-0190-deprecations'></a>

## Deprecations

- `Series.reshape` and `Categorical.reshape` have been deprecated and will be removed in a subsequent release ([GH12882](https://github.com/pandas-dev/pandas/issues/12882), [GH12882](https://github.com/pandas-dev/pandas/issues/12882))  
- `PeriodIndex.to_datetime` has been deprecated in favor of `PeriodIndex.to_timestamp` ([GH8254](https://github.com/pandas-dev/pandas/issues/8254))  
- `Timestamp.to_datetime` has been deprecated in favor of `Timestamp.to_pydatetime` ([GH8254](https://github.com/pandas-dev/pandas/issues/8254))  
- `Index.to_datetime` and `DatetimeIndex.to_datetime` have been deprecated in favor of `pd.to_datetime` ([GH8254](https://github.com/pandas-dev/pandas/issues/8254))  
- `pandas.core.datetools` module has been deprecated and will be removed in a subsequent release ([GH14094](https://github.com/pandas-dev/pandas/issues/14094))  
- `SparseList` has been deprecated and will be removed in a future version ([GH13784](https://github.com/pandas-dev/pandas/issues/13784))  
- `DataFrame.to_html()` and `DataFrame.to_latex()` have dropped the `colSpace` parameter in favor of `col_space` ([GH13857](https://github.com/pandas-dev/pandas/issues/13857))  
- `DataFrame.to_sql()` has deprecated the `flavor` parameter, as it is superfluous when SQLAlchemy is not installed ([GH13611](https://github.com/pandas-dev/pandas/issues/13611))  
- Deprecated `read_csv` keywords:  
  - `compact_ints` and `use_unsigned` have been deprecated and will be removed in a future version ([GH13320](https://github.com/pandas-dev/pandas/issues/13320))  
  - `buffer_lines` has been deprecated and will be removed in a future version ([GH13360](https://github.com/pandas-dev/pandas/issues/13360))  
  - `as_recarray` has been deprecated and will be removed in a future version ([GH13373](https://github.com/pandas-dev/pandas/issues/13373))  
  - `skip_footer` has been deprecated in favor of `skipfooter` and will be removed in a future version ([GH13349](https://github.com/pandas-dev/pandas/issues/13349))  
- top-level `pd.ordered_merge()` has been renamed to `pd.merge_ordered()` and the original name will be removed in a future version ([GH13358](https://github.com/pandas-dev/pandas/issues/13358))  
- `Timestamp.offset` property (and named arg in the constructor), has been deprecated in favor of `freq` ([GH12160](https://github.com/pandas-dev/pandas/issues/12160))  
- `pd.tseries.util.pivot_annual` is deprecated. Use `pivot_table` as alternative, an example is [here](user_guide/cookbook.ipynb#cookbook-pivot) ([GH736](https://github.com/pandas-dev/pandas/issues/736))  
- `pd.tseries.util.isleapyear` has been deprecated and will be removed in a subsequent release. Datetime-likes now have a `.is_leap_year` property ([GH13727](https://github.com/pandas-dev/pandas/issues/13727))  
- `Panel4D` and `PanelND` constructors are deprecated and will be removed in a future version. The recommended way to represent these types of n-dimensional data are with the [xarray package](http://xarray.pydata.org/en/stable/). Pandas provides a `to_xarray()` method to automate this conversion ([GH13564](https://github.com/pandas-dev/pandas/issues/13564)).  
- `pandas.tseries.frequencies.get_standard_freq` is deprecated. Use  `pandas.tseries.frequencies.to_offset(freq).rule_code` instead ([GH13874](https://github.com/pandas-dev/pandas/issues/13874))  
- `pandas.tseries.frequencies.to_offset`’s `freqstr` keyword is deprecated in favor of `freq` ([GH13874](https://github.com/pandas-dev/pandas/issues/13874))  
- `Categorical.from_array` has been deprecated and will be removed in a future version ([GH13854](https://github.com/pandas-dev/pandas/issues/13854))  



<a id='whatsnew-0190-prior-deprecations'></a>

## Removal of prior version deprecations/changes

- The `SparsePanel` class has been removed ([GH13778](https://github.com/pandas-dev/pandas/issues/13778))  
- The `pd.sandbox` module has been removed in favor of the external library `pandas-qt` ([GH13670](https://github.com/pandas-dev/pandas/issues/13670))  
- The `pandas.io.data` and `pandas.io.wb` modules are removed in favor of
  the [pandas-datareader package](https://github.com/pydata/pandas-datareader) ([GH13724](https://github.com/pandas-dev/pandas/issues/13724)).  
- The `pandas.tools.rplot` module has been removed in favor of
  the [seaborn package](https://github.com/mwaskom/seaborn) ([GH13855](https://github.com/pandas-dev/pandas/issues/13855))  
- `DataFrame.to_csv()` has dropped the `engine` parameter, as was deprecated in 0.17.1 ([GH11274](https://github.com/pandas-dev/pandas/issues/11274), [GH13419](https://github.com/pandas-dev/pandas/issues/13419))  
- `DataFrame.to_dict()` has dropped the `outtype` parameter in favor of `orient` ([GH13627](https://github.com/pandas-dev/pandas/issues/13627), [GH8486](https://github.com/pandas-dev/pandas/issues/8486))  
- `pd.Categorical` has dropped setting of the `ordered` attribute directly in favor of the `set_ordered` method ([GH13671](https://github.com/pandas-dev/pandas/issues/13671))  
- `pd.Categorical` has dropped the `levels` attribute in favor of `categories` ([GH8376](https://github.com/pandas-dev/pandas/issues/8376))  
- `DataFrame.to_sql()` has dropped the `mysql` option for the `flavor` parameter ([GH13611](https://github.com/pandas-dev/pandas/issues/13611))  
- `Panel.shift()` has dropped the `lags` parameter in favor of `periods` ([GH14041](https://github.com/pandas-dev/pandas/issues/14041))  
- `pd.Index` has dropped the `diff` method in favor of `difference` ([GH13669](https://github.com/pandas-dev/pandas/issues/13669))  
- `pd.DataFrame` has dropped the `to_wide` method in favor of `to_panel` ([GH14039](https://github.com/pandas-dev/pandas/issues/14039))  
- `Series.to_csv` has dropped the `nanRep` parameter in favor of `na_rep` ([GH13804](https://github.com/pandas-dev/pandas/issues/13804))  
- `Series.xs`, `DataFrame.xs`, `Panel.xs`, `Panel.major_xs`, and `Panel.minor_xs` have dropped the `copy` parameter ([GH13781](https://github.com/pandas-dev/pandas/issues/13781))  
- `str.split` has dropped the `return_type` parameter in favor of `expand` ([GH13701](https://github.com/pandas-dev/pandas/issues/13701))  
- Removal of the legacy time rules (offset aliases), deprecated since 0.17.0 (this has been alias since 0.8.0) ([GH13590](https://github.com/pandas-dev/pandas/issues/13590), [GH13868](https://github.com/pandas-dev/pandas/issues/13868)). Now legacy time rules raises `ValueError`. For the list of currently supported offsets, see [here](user_guide/timeseries.ipynb#timeseries-offset-aliases).  
- The default value for the `return_type` parameter for `DataFrame.plot.box` and `DataFrame.boxplot` changed from `None` to `"axes"`. These methods will now return a matplotlib axes by default instead of a dictionary of artists. See [here](user_guide/visualization.ipynb#visualization-box-return) ([GH6581](https://github.com/pandas-dev/pandas/issues/6581)).  
- The `tquery` and `uquery` functions in the `pandas.io.sql` module are removed ([GH5950](https://github.com/pandas-dev/pandas/issues/5950)).  



<a id='whatsnew-0190-performance'></a>

## Performance improvements

- Improved performance of sparse `IntIndex.intersect` ([GH13082](https://github.com/pandas-dev/pandas/issues/13082))  
- Improved performance of sparse arithmetic with `BlockIndex` when the number of blocks are large, though recommended to use `IntIndex` in such cases ([GH13082](https://github.com/pandas-dev/pandas/issues/13082))  
- Improved performance of `DataFrame.quantile()` as it now operates per-block ([GH11623](https://github.com/pandas-dev/pandas/issues/11623))  
- Improved performance of float64 hash table operations, fixing some very slow indexing and groupby operations in python 3 ([GH13166](https://github.com/pandas-dev/pandas/issues/13166), [GH13334](https://github.com/pandas-dev/pandas/issues/13334))  
- Improved performance of `DataFrameGroupBy.transform` ([GH12737](https://github.com/pandas-dev/pandas/issues/12737))  
- Improved performance of `Index` and `Series` `.duplicated` ([GH10235](https://github.com/pandas-dev/pandas/issues/10235))  
- Improved performance of `Index.difference` ([GH12044](https://github.com/pandas-dev/pandas/issues/12044))  
- Improved performance of `RangeIndex.is_monotonic_increasing` and `is_monotonic_decreasing` ([GH13749](https://github.com/pandas-dev/pandas/issues/13749))  
- Improved performance of datetime string parsing in `DatetimeIndex` ([GH13692](https://github.com/pandas-dev/pandas/issues/13692))  
- Improved performance of hashing `Period` ([GH12817](https://github.com/pandas-dev/pandas/issues/12817))  
- Improved performance of `factorize` of datetime with timezone ([GH13750](https://github.com/pandas-dev/pandas/issues/13750))  
- Improved performance of by lazily creating indexing hashtables on larger Indexes ([GH14266](https://github.com/pandas-dev/pandas/issues/14266))  
- Improved performance of `groupby.groups` ([GH14293](https://github.com/pandas-dev/pandas/issues/14293))  
- Unnecessary materializing of a MultiIndex when introspecting for memory usage ([GH14308](https://github.com/pandas-dev/pandas/issues/14308))  



<a id='whatsnew-0190-bug-fixes'></a>

## Bug fixes

- Bug in `groupby().shift()`, which could cause a segfault or corruption in rare circumstances when grouping by columns with missing values ([GH13813](https://github.com/pandas-dev/pandas/issues/13813))  
- Bug in `groupby().cumsum()` calculating `cumprod` when `axis=1`. ([GH13994](https://github.com/pandas-dev/pandas/issues/13994))  
- Bug in `pd.to_timedelta()` in which the `errors` parameter was not being respected ([GH13613](https://github.com/pandas-dev/pandas/issues/13613))  
- Bug in `io.json.json_normalize()`, where non-ascii keys raised an exception ([GH13213](https://github.com/pandas-dev/pandas/issues/13213))  
- Bug when passing a not-default-indexed `Series` as `xerr` or `yerr` in `.plot()` ([GH11858](https://github.com/pandas-dev/pandas/issues/11858))  
- Bug in area plot draws legend incorrectly if subplot is enabled or legend is moved after plot (matplotlib 1.5.0 is required to draw area plot legend properly) ([GH9161](https://github.com/pandas-dev/pandas/issues/9161), [GH13544](https://github.com/pandas-dev/pandas/issues/13544))  
- Bug in `DataFrame` assignment with an object-dtyped `Index` where the resultant column is mutable to the original object. ([GH13522](https://github.com/pandas-dev/pandas/issues/13522))  
- Bug in matplotlib `AutoDataFormatter`; this restores the second scaled formatting and re-adds micro-second scaled formatting ([GH13131](https://github.com/pandas-dev/pandas/issues/13131))  
- Bug in selection from a `HDFStore` with a fixed format and `start` and/or `stop` specified will now return the selected range ([GH8287](https://github.com/pandas-dev/pandas/issues/8287))  
- Bug in `Categorical.from_codes()` where an unhelpful error was raised when an invalid `ordered` parameter was passed in ([GH14058](https://github.com/pandas-dev/pandas/issues/14058))  
- Bug in `Series` construction from a tuple of integers on windows not returning default dtype (int64) ([GH13646](https://github.com/pandas-dev/pandas/issues/13646))  
- Bug in `TimedeltaIndex` addition with a Datetime-like object where addition overflow was not being caught ([GH14068](https://github.com/pandas-dev/pandas/issues/14068))  
- Bug in `.groupby(..).resample(..)` when the same object is called multiple times ([GH13174](https://github.com/pandas-dev/pandas/issues/13174))  
- Bug in `.to_records()` when index name is a unicode string ([GH13172](https://github.com/pandas-dev/pandas/issues/13172))  
- Bug in calling `.memory_usage()` on object which doesn’t implement ([GH12924](https://github.com/pandas-dev/pandas/issues/12924))  
- Regression in `Series.quantile` with nans (also shows up in `.median()` and `.describe()` ); furthermore now names the `Series` with the quantile ([GH13098](https://github.com/pandas-dev/pandas/issues/13098), [GH13146](https://github.com/pandas-dev/pandas/issues/13146))  
- Bug in `SeriesGroupBy.transform` with datetime values and missing groups ([GH13191](https://github.com/pandas-dev/pandas/issues/13191))  
- Bug where empty `Series` were incorrectly coerced in datetime-like numeric operations ([GH13844](https://github.com/pandas-dev/pandas/issues/13844))  
- Bug in `Categorical` constructor when passed a `Categorical` containing datetimes with timezones ([GH14190](https://github.com/pandas-dev/pandas/issues/14190))  
- Bug in `Series.str.extractall()` with `str` index raises `ValueError`  ([GH13156](https://github.com/pandas-dev/pandas/issues/13156))  
- Bug in `Series.str.extractall()` with single group and quantifier  ([GH13382](https://github.com/pandas-dev/pandas/issues/13382))  
- Bug in `DatetimeIndex` and `Period` subtraction raises `ValueError` or `AttributeError` rather than `TypeError` ([GH13078](https://github.com/pandas-dev/pandas/issues/13078))  
- Bug in `Index` and `Series` created with `NaN` and `NaT` mixed data may not have `datetime64` dtype  ([GH13324](https://github.com/pandas-dev/pandas/issues/13324))  
- Bug in `Index` and `Series` may ignore `np.datetime64('nat')` and `np.timdelta64('nat')` to infer dtype ([GH13324](https://github.com/pandas-dev/pandas/issues/13324))  
- Bug in `PeriodIndex` and `Period` subtraction raises `AttributeError` ([GH13071](https://github.com/pandas-dev/pandas/issues/13071))  
- Bug in `PeriodIndex` construction returning a `float64` index in some circumstances ([GH13067](https://github.com/pandas-dev/pandas/issues/13067))  
- Bug in `.resample(..)` with a `PeriodIndex` not changing its `freq` appropriately when empty ([GH13067](https://github.com/pandas-dev/pandas/issues/13067))  
- Bug in `.resample(..)` with a `PeriodIndex` not retaining its type or name with an empty `DataFrame` appropriately when empty ([GH13212](https://github.com/pandas-dev/pandas/issues/13212))  
- Bug in `groupby(..).apply(..)` when the passed function returns scalar values per group ([GH13468](https://github.com/pandas-dev/pandas/issues/13468)).  
- Bug in `groupby(..).resample(..)` where passing some keywords would raise an exception ([GH13235](https://github.com/pandas-dev/pandas/issues/13235))  
- Bug in `.tz_convert` on a tz-aware `DateTimeIndex` that relied on index being sorted for correct results ([GH13306](https://github.com/pandas-dev/pandas/issues/13306))  
- Bug in `.tz_localize` with `dateutil.tz.tzlocal` may return incorrect result ([GH13583](https://github.com/pandas-dev/pandas/issues/13583))  
- Bug in `DatetimeTZDtype` dtype with `dateutil.tz.tzlocal` cannot be regarded as valid dtype ([GH13583](https://github.com/pandas-dev/pandas/issues/13583))  
- Bug in `pd.read_hdf()` where attempting to load an HDF file with a single dataset, that had one or more categorical columns, failed unless the key argument was set to the name of the dataset. ([GH13231](https://github.com/pandas-dev/pandas/issues/13231))  
- Bug in `.rolling()` that allowed a negative integer window in construction of the `Rolling()` object, but would later fail on aggregation ([GH13383](https://github.com/pandas-dev/pandas/issues/13383))  
- Bug in `Series` indexing with tuple-valued data and a numeric index ([GH13509](https://github.com/pandas-dev/pandas/issues/13509))  
- Bug in printing `pd.DataFrame` where unusual elements with the `object` dtype were causing segfaults ([GH13717](https://github.com/pandas-dev/pandas/issues/13717))  
- Bug in ranking `Series` which could result in segfaults ([GH13445](https://github.com/pandas-dev/pandas/issues/13445))  
- Bug in various index types, which did not propagate the name of passed index ([GH12309](https://github.com/pandas-dev/pandas/issues/12309))  
- Bug in `DatetimeIndex`, which did not honour the `copy=True` ([GH13205](https://github.com/pandas-dev/pandas/issues/13205))  
- Bug in `DatetimeIndex.is_normalized` returns incorrectly for normalized date_range in case of local timezones ([GH13459](https://github.com/pandas-dev/pandas/issues/13459))  
- Bug in `pd.concat` and `.append` may coerces `datetime64` and `timedelta` to `object` dtype containing python built-in `datetime` or `timedelta` rather than `Timestamp` or `Timedelta` ([GH13626](https://github.com/pandas-dev/pandas/issues/13626))  
- Bug in `PeriodIndex.append` may raises `AttributeError` when the result is `object` dtype ([GH13221](https://github.com/pandas-dev/pandas/issues/13221))  
- Bug in `CategoricalIndex.append` may accept normal `list` ([GH13626](https://github.com/pandas-dev/pandas/issues/13626))  
- Bug in `pd.concat` and `.append` with the same timezone get reset to UTC ([GH7795](https://github.com/pandas-dev/pandas/issues/7795))  
- Bug in `Series` and `DataFrame` `.append` raises `AmbiguousTimeError` if data contains datetime near DST boundary ([GH13626](https://github.com/pandas-dev/pandas/issues/13626))  
- Bug in `DataFrame.to_csv()` in which float values were being quoted even though quotations were specified for non-numeric values only ([GH12922](https://github.com/pandas-dev/pandas/issues/12922), [GH13259](https://github.com/pandas-dev/pandas/issues/13259))  
- Bug in `DataFrame.describe()` raising `ValueError` with only boolean columns ([GH13898](https://github.com/pandas-dev/pandas/issues/13898))  
- Bug in `MultiIndex` slicing where extra elements were returned when level is non-unique ([GH12896](https://github.com/pandas-dev/pandas/issues/12896))  
- Bug in `.str.replace` does not raise `TypeError` for invalid replacement ([GH13438](https://github.com/pandas-dev/pandas/issues/13438))  
- Bug in `MultiIndex.from_arrays` which didn’t check for input array lengths matching ([GH13599](https://github.com/pandas-dev/pandas/issues/13599))  
- Bug in `cartesian_product` and `MultiIndex.from_product` which may raise with empty input arrays ([GH12258](https://github.com/pandas-dev/pandas/issues/12258))  
- Bug in `pd.read_csv()` which may cause a segfault or corruption when iterating in large chunks over a stream/file under rare circumstances ([GH13703](https://github.com/pandas-dev/pandas/issues/13703))  
- Bug in `pd.read_csv()` which caused errors to be raised when a dictionary containing scalars is passed in for `na_values` ([GH12224](https://github.com/pandas-dev/pandas/issues/12224))  
- Bug in `pd.read_csv()` which caused BOM files to be incorrectly parsed by not ignoring the BOM ([GH4793](https://github.com/pandas-dev/pandas/issues/4793))  
- Bug in `pd.read_csv()` with `engine='python'` which raised errors when a numpy array was passed in for `usecols` ([GH12546](https://github.com/pandas-dev/pandas/issues/12546))  
- Bug in `pd.read_csv()` where the index columns were being incorrectly parsed when parsed as dates with a `thousands` parameter ([GH14066](https://github.com/pandas-dev/pandas/issues/14066))  
- Bug in `pd.read_csv()` with `engine='python'` in which `NaN` values weren’t being detected after data was converted to numeric values ([GH13314](https://github.com/pandas-dev/pandas/issues/13314))  
- Bug in `pd.read_csv()` in which the `nrows` argument was not properly validated for both engines ([GH10476](https://github.com/pandas-dev/pandas/issues/10476))  
- Bug in `pd.read_csv()` with `engine='python'` in which infinities of mixed-case forms were not being interpreted properly ([GH13274](https://github.com/pandas-dev/pandas/issues/13274))  
- Bug in `pd.read_csv()` with `engine='python'` in which trailing `NaN` values were not being parsed ([GH13320](https://github.com/pandas-dev/pandas/issues/13320))  
- Bug in `pd.read_csv()` with `engine='python'` when reading from a `tempfile.TemporaryFile` on Windows with Python 3 ([GH13398](https://github.com/pandas-dev/pandas/issues/13398))  
- Bug in `pd.read_csv()` that prevents `usecols` kwarg from accepting single-byte unicode strings ([GH13219](https://github.com/pandas-dev/pandas/issues/13219))  
- Bug in `pd.read_csv()` that prevents `usecols` from being an empty set ([GH13402](https://github.com/pandas-dev/pandas/issues/13402))  
- Bug in `pd.read_csv()` in the C engine where the NULL character was not being parsed as NULL ([GH14012](https://github.com/pandas-dev/pandas/issues/14012))  
- Bug in `pd.read_csv()` with `engine='c'` in which NULL `quotechar` was not accepted even though `quoting` was specified as `None` ([GH13411](https://github.com/pandas-dev/pandas/issues/13411))  
- Bug in `pd.read_csv()` with `engine='c'` in which fields were not properly cast to float when quoting was specified as non-numeric ([GH13411](https://github.com/pandas-dev/pandas/issues/13411))  
- Bug in `pd.read_csv()` in Python 2.x with non-UTF8 encoded, multi-character separated data ([GH3404](https://github.com/pandas-dev/pandas/issues/3404))  
- Bug in `pd.read_csv()`, where aliases for utf-xx (e.g. UTF-xx, UTF_xx, utf_xx) raised UnicodeDecodeError ([GH13549](https://github.com/pandas-dev/pandas/issues/13549))  
- Bug in `pd.read_csv`, `pd.read_table`, `pd.read_fwf`, `pd.read_stata` and `pd.read_sas` where files were opened by parsers but not closed if both `chunksize` and `iterator` were `None`. ([GH13940](https://github.com/pandas-dev/pandas/issues/13940))  
- Bug in `StataReader`, `StataWriter`, `XportReader` and `SAS7BDATReader` where a file was not properly closed when an error was raised. ([GH13940](https://github.com/pandas-dev/pandas/issues/13940))  
- Bug in `pd.pivot_table()` where `margins_name` is ignored when `aggfunc` is a list ([GH13354](https://github.com/pandas-dev/pandas/issues/13354))  
- Bug in `pd.Series.str.zfill`, `center`, `ljust`, `rjust`, and `pad` when passing non-integers, did not raise `TypeError` ([GH13598](https://github.com/pandas-dev/pandas/issues/13598))  
- Bug in checking for any null objects in a `TimedeltaIndex`, which always returned `True` ([GH13603](https://github.com/pandas-dev/pandas/issues/13603))  
- Bug in `Series` arithmetic raises `TypeError` if it contains datetime-like as `object` dtype ([GH13043](https://github.com/pandas-dev/pandas/issues/13043))  
- Bug `Series.isnull()` and `Series.notnull()` ignore `Period('NaT')`  ([GH13737](https://github.com/pandas-dev/pandas/issues/13737))  
- Bug `Series.fillna()` and `Series.dropna()` don’t affect to `Period('NaT')`  ([GH13737](https://github.com/pandas-dev/pandas/issues/13737)  
- Bug in `.fillna(value=np.nan)` incorrectly raises `KeyError` on a `category` dtyped `Series` ([GH14021](https://github.com/pandas-dev/pandas/issues/14021))  
- Bug in extension dtype creation where the created types were not is/identical ([GH13285](https://github.com/pandas-dev/pandas/issues/13285))  
- Bug in `.resample(..)` where incorrect warnings were triggered by IPython introspection ([GH13618](https://github.com/pandas-dev/pandas/issues/13618))  
- Bug in `NaT` - `Period` raises `AttributeError` ([GH13071](https://github.com/pandas-dev/pandas/issues/13071))  
- Bug in `Series` comparison may output incorrect result if rhs contains `NaT` ([GH9005](https://github.com/pandas-dev/pandas/issues/9005))  
- Bug in `Series` and `Index` comparison may output incorrect result if it contains `NaT` with `object` dtype ([GH13592](https://github.com/pandas-dev/pandas/issues/13592))  
- Bug in `Period` addition raises `TypeError` if `Period` is on right hand side ([GH13069](https://github.com/pandas-dev/pandas/issues/13069))  
- Bug in `Period` and `Series` or `Index` comparison raises `TypeError` ([GH13200](https://github.com/pandas-dev/pandas/issues/13200))  
- Bug in `pd.set_eng_float_format()` that would prevent NaN and Inf from formatting ([GH11981](https://github.com/pandas-dev/pandas/issues/11981))  
- Bug in `.unstack` with `Categorical` dtype resets `.ordered` to `True` ([GH13249](https://github.com/pandas-dev/pandas/issues/13249))  
- Clean some compile time warnings in datetime parsing ([GH13607](https://github.com/pandas-dev/pandas/issues/13607))  
- Bug in `factorize` raises `AmbiguousTimeError` if data contains datetime near DST boundary ([GH13750](https://github.com/pandas-dev/pandas/issues/13750))  
- Bug in `.set_index` raises `AmbiguousTimeError` if new index contains DST boundary and multi levels ([GH12920](https://github.com/pandas-dev/pandas/issues/12920))  
- Bug in `.shift` raises `AmbiguousTimeError` if data contains datetime near DST boundary ([GH13926](https://github.com/pandas-dev/pandas/issues/13926))  
- Bug in `pd.read_hdf()` returns incorrect result when a `DataFrame` with a `categorical` column and a query which doesn’t match any values ([GH13792](https://github.com/pandas-dev/pandas/issues/13792))  
- Bug in `.iloc` when indexing with a non lexsorted MultiIndex ([GH13797](https://github.com/pandas-dev/pandas/issues/13797))  
- Bug in `.loc` when indexing with date strings in a reverse sorted `DatetimeIndex` ([GH14316](https://github.com/pandas-dev/pandas/issues/14316))  
- Bug in `Series` comparison operators when dealing with zero dim NumPy arrays ([GH13006](https://github.com/pandas-dev/pandas/issues/13006))  
- Bug in `.combine_first` may return incorrect `dtype` ([GH7630](https://github.com/pandas-dev/pandas/issues/7630), [GH10567](https://github.com/pandas-dev/pandas/issues/10567))  
- Bug in `groupby` where `apply` returns different result depending on whether first result is `None` or not ([GH12824](https://github.com/pandas-dev/pandas/issues/12824))  
- Bug in `groupby(..).nth()` where the group key is included inconsistently if called after `.head()/.tail()` ([GH12839](https://github.com/pandas-dev/pandas/issues/12839))  
- Bug in `.to_html`, `.to_latex` and `.to_string` silently ignore custom datetime formatter passed through the `formatters` key word ([GH10690](https://github.com/pandas-dev/pandas/issues/10690))  
- Bug in `DataFrame.iterrows()`, not yielding a `Series` subclasse if defined ([GH13977](https://github.com/pandas-dev/pandas/issues/13977))  
- Bug in `pd.to_numeric` when `errors='coerce'` and input contains non-hashable objects ([GH13324](https://github.com/pandas-dev/pandas/issues/13324))  
- Bug in invalid `Timedelta` arithmetic and comparison may raise `ValueError` rather than `TypeError` ([GH13624](https://github.com/pandas-dev/pandas/issues/13624))  
- Bug in invalid datetime parsing in `to_datetime` and `DatetimeIndex` may raise `TypeError` rather than `ValueError` ([GH11169](https://github.com/pandas-dev/pandas/issues/11169), [GH11287](https://github.com/pandas-dev/pandas/issues/11287))  
- Bug in `Index` created with tz-aware `Timestamp` and mismatched `tz` option incorrectly coerces timezone ([GH13692](https://github.com/pandas-dev/pandas/issues/13692))  
- Bug in `DatetimeIndex` with nanosecond frequency does not include timestamp specified with `end` ([GH13672](https://github.com/pandas-dev/pandas/issues/13672))  
- Bug in ``Series` when setting a slice with a `np.timedelta64` ([GH14155](https://github.com/pandas-dev/pandas/issues/14155))  
- Bug in `Index` raises `OutOfBoundsDatetime` if `datetime` exceeds `datetime64[ns]` bounds, rather than coercing to `object` dtype ([GH13663](https://github.com/pandas-dev/pandas/issues/13663))  
- Bug in `Index` may ignore specified `datetime64` or `timedelta64` passed as `dtype`  ([GH13981](https://github.com/pandas-dev/pandas/issues/13981))  
- Bug in `RangeIndex` can be created without no arguments rather than raises `TypeError` ([GH13793](https://github.com/pandas-dev/pandas/issues/13793))  
- Bug in `.value_counts()` raises `OutOfBoundsDatetime` if data exceeds `datetime64[ns]` bounds ([GH13663](https://github.com/pandas-dev/pandas/issues/13663))  
- Bug in `DatetimeIndex` may raise `OutOfBoundsDatetime` if input `np.datetime64` has other unit than `ns` ([GH9114](https://github.com/pandas-dev/pandas/issues/9114))  
- Bug in `Series` creation with `np.datetime64` which has other unit than `ns` as `object` dtype results in incorrect values ([GH13876](https://github.com/pandas-dev/pandas/issues/13876))  
- Bug in `resample` with timedelta data where data was casted to float ([GH13119](https://github.com/pandas-dev/pandas/issues/13119)).  
- Bug in `pd.isnull()` `pd.notnull()` raise `TypeError` if input datetime-like has other unit than `ns` ([GH13389](https://github.com/pandas-dev/pandas/issues/13389))  
- Bug in `pd.merge()` may raise `TypeError` if input datetime-like has other unit than `ns` ([GH13389](https://github.com/pandas-dev/pandas/issues/13389))  
- Bug in `HDFStore`/`read_hdf()` discarded `DatetimeIndex.name` if `tz` was set ([GH13884](https://github.com/pandas-dev/pandas/issues/13884))  
- Bug in `Categorical.remove_unused_categories()` changes `.codes` dtype to platform int ([GH13261](https://github.com/pandas-dev/pandas/issues/13261))  
- Bug in `groupby` with `as_index=False` returns all NaN’s when grouping on multiple columns including a categorical one ([GH13204](https://github.com/pandas-dev/pandas/issues/13204))  
- Bug in `df.groupby(...)[...]` where getitem with `Int64Index` raised an error ([GH13731](https://github.com/pandas-dev/pandas/issues/13731))  
- Bug in the CSS classes assigned to `DataFrame.style` for index names. Previously they were assigned `"col_heading level<n> col<c>"` where `n` was the number of levels + 1. Now they are assigned `"index_name level<n>"`, where `n` is the correct level for that MultiIndex.  
- Bug where `pd.read_gbq()` could throw `ImportError: No module named discovery` as a result of a naming conflict with another python package called apiclient  ([GH13454](https://github.com/pandas-dev/pandas/issues/13454))  
- Bug in `Index.union` returns an incorrect result with a named empty index ([GH13432](https://github.com/pandas-dev/pandas/issues/13432))  
- Bugs in `Index.difference` and `DataFrame.join` raise in Python3 when using mixed-integer indexes ([GH13432](https://github.com/pandas-dev/pandas/issues/13432), [GH12814](https://github.com/pandas-dev/pandas/issues/12814))  
- Bug in subtract tz-aware `datetime.datetime` from tz-aware `datetime64` series ([GH14088](https://github.com/pandas-dev/pandas/issues/14088))  
- Bug in `.to_excel()` when DataFrame contains a MultiIndex which contains a label with a NaN value ([GH13511](https://github.com/pandas-dev/pandas/issues/13511))  
- Bug in invalid frequency offset string like “D1”, “-2-3H” may not raise `ValueError` ([GH13930](https://github.com/pandas-dev/pandas/issues/13930))  
- Bug in `concat` and `groupby` for hierarchical frames with `RangeIndex` levels ([GH13542](https://github.com/pandas-dev/pandas/issues/13542)).  
- Bug in `Series.str.contains()` for Series containing only `NaN` values of `object` dtype ([GH14171](https://github.com/pandas-dev/pandas/issues/14171))  
- Bug in `agg()` function on groupby dataframe changes dtype of `datetime64[ns]` column to `float64` ([GH12821](https://github.com/pandas-dev/pandas/issues/12821))  
- Bug in using NumPy ufunc with `PeriodIndex` to add or subtract integer raise `IncompatibleFrequency`. Note that using standard operator like `+` or `-` is recommended, because standard operators use more efficient path ([GH13980](https://github.com/pandas-dev/pandas/issues/13980))  
- Bug in operations on `NaT` returning `float` instead of `datetime64[ns]` ([GH12941](https://github.com/pandas-dev/pandas/issues/12941))  
- Bug in `Series` flexible arithmetic methods (like `.add()`) raises `ValueError` when `axis=None` ([GH13894](https://github.com/pandas-dev/pandas/issues/13894))  
- Bug in `DataFrame.to_csv()` with `MultiIndex` columns in which a stray empty line was added ([GH6618](https://github.com/pandas-dev/pandas/issues/6618))  
- Bug in `DatetimeIndex`, `TimedeltaIndex` and `PeriodIndex.equals()` may return `True` when input isn’t `Index` but contains the same values ([GH13107](https://github.com/pandas-dev/pandas/issues/13107))  
- Bug in assignment against datetime with timezone may not work if it contains datetime near DST boundary ([GH14146](https://github.com/pandas-dev/pandas/issues/14146))  
- Bug in `pd.eval()` and `HDFStore` query truncating long float literals with python 2 ([GH14241](https://github.com/pandas-dev/pandas/issues/14241))  
- Bug in `Index` raises `KeyError` displaying incorrect column when column is not in the df and columns contains duplicate values ([GH13822](https://github.com/pandas-dev/pandas/issues/13822))  
- Bug in `Period` and `PeriodIndex` creating wrong dates when frequency has combined offset aliases ([GH13874](https://github.com/pandas-dev/pandas/issues/13874))  
- Bug in `.to_string()` when called with an integer `line_width` and `index=False` raises an UnboundLocalError exception because `idx` referenced before assignment.  
- Bug in `eval()` where the `resolvers` argument would not accept a list ([GH14095](https://github.com/pandas-dev/pandas/issues/14095))  
- Bugs in `stack`, `get_dummies`, `make_axis_dummies` which don’t preserve categorical dtypes in (multi)indexes ([GH13854](https://github.com/pandas-dev/pandas/issues/13854))  
- `PeriodIndex` can now accept `list` and `array` which contains `pd.NaT` ([GH13430](https://github.com/pandas-dev/pandas/issues/13430))  
- Bug in `df.groupby` where `.median()` returns arbitrary values if grouped dataframe contains empty bins ([GH13629](https://github.com/pandas-dev/pandas/issues/13629))  
- Bug in `Index.copy()` where `name` parameter was ignored ([GH14302](https://github.com/pandas-dev/pandas/issues/14302))  



<a id='whatsnew-0-19-0-contributors'></a>

## Contributors

A total of 117 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Adrien Emery +  
- Alex Alekseyev  
- Alex Vig +  
- Allen Riddell +  
- Amol +  
- Amol Agrawal +  
- Andy R. Terrel +  
- Anthonios Partheniou  
- Ben Kandel +  
- Bob Baxley +  
- Brett Rosen +  
- Camilo Cota +  
- Chris  
- Chris Grinolds  
- Chris Warth  
- Christian Hudon  
- Christopher C. Aycock  
- Daniel Siladji +  
- Douglas McNeil  
- Drewrey Lupton +  
- Eduardo Blancas Reyes +  
- Elliot Marsden +  
- Evan Wright  
- Felix Marczinowski +  
- Francis T. O’Donovan  
- Geraint Duck +  
- Giacomo Ferroni +  
- Grant Roch +  
- Gábor Lipták  
- Haleemur Ali +  
- Hassan Shamim +  
- Iulius Curt +  
- Ivan Nazarov +  
- Jeff Reback  
- Jeffrey Gerard +  
- Jenn Olsen +  
- Jim Crist  
- Joe Jevnik  
- John Evans +  
- John Freeman  
- John Liekezer +  
- John W. O’Brien  
- John Zwinck +  
- Johnny Gill +  
- Jordan Erenrich +  
- Joris Van den Bossche  
- Josh Howes +  
- Jozef Brandys +  
- Ka Wo Chen  
- Kamil Sindi +  
- Kerby Shedden  
- Kernc +  
- Kevin Sheppard  
- Matthieu Brucher +  
- Maximilian Roos  
- Michael Scherer +  
- Mike Graham +  
- Mortada Mehyar  
- Muhammad Haseeb Tariq +  
- Nate George +  
- Neil Parley +  
- Nicolas Bonnotte  
- OXPHOS  
- Pan Deng / Zora +  
- Paul +  
- Paul Mestemaker +  
- Pauli Virtanen  
- Pawel Kordek +  
- Pietro Battiston  
- Piotr Jucha +  
- Ravi Kumar Nimmi +  
- Robert Gieseke  
- Robert Kern +  
- Roger Thomas  
- Roy Keyes +  
- Russell Smith +  
- Sahil Dua +  
- Sanjiv Lobo +  
- Sašo Stanovnik +  
- Shawn Heide +  
- Sinhrks  
- Stephen Kappel +  
- Steve Choi +  
- Stewart Henderson +  
- Sudarshan Konge +  
- Thomas A Caswell  
- Tom Augspurger  
- Tom Bird +  
- Uwe Hoffmann +  
- WillAyd +  
- Xiang Zhang +  
- YG-Riku +  
- Yadunandan +  
- Yaroslav Halchenko  
- Yuichiro Kaneko +  
- adneu  
- agraboso +  
- babakkeyvani +  
- c123w +  
- chris-b1  
- cmazzullo +  
- conquistador1492 +  
- cr3 +  
- dsm054  
- gfyoung  
- harshul1610 +  
- iamsimha +  
- jackieleng +  
- mpuels +  
- pijucha +  
- priyankjain +  
- sinhrks  
- wcwagner +  
- yui-knk +  
- zhangjinjie +  
- znmean +  
- 颜发才（Yan Facai） +  